In [ ]:
import chromadb
from chromadb.config import Settings
import pandas as pd

In [ ]:
# vector store settings
VECTOR_STORE_PATH = r'./00_Vector_Store'
COLLECTION_NAME = 'my_collection'

embeddings_df = pd.read_csv('./embeddings_df.csv')

In [ ]:
def get_or_create_client_and_collection(VECTOR_STORE_PATH, COLLECTION_NAME):
    chroma_client = chromadb.PersistentClient(path=VECTOR_STORE_PATH)

    collection = chroma_client.get_or_create_collection(name=COLLECTION_NAME)

    return collection

collection = get_or_create_client_and_collection(VECTOR_STORE_PATH, COLLECTION_NAME)

In [ ]:
def add_to_collection(embeddings_df):
    embeddings_df['embeddings_array'] = embeddings_df.apply(lambda row: row.values[:-1], axis=1)
    embeddings_df['embeddings_array'] = embeddings_df['embeddings_array'].apply(lambda x: x.tolist())

    collection.add(
        embeddings=embeddings_df.embeddings_array.to_list(),
        documents=embeddings_df.text_chunk.to_list(),
        ids=list(map(str, embeddings_df.index.tolist()))
    )


In [ ]:
# add the embeddings_df to our vector store collection
add_to_collection(embeddings_df)

In [ ]:
def get_all_entries(collection):
    # query collection
    existing_docs = pd.DataFrame(collection.get()).rename(columns={0: "ids", 1:"embeddings", 2:"documents", 3:"metadatas"})
    existing_docs.to_excel(r"./01_vector_stores_export.xlsx")
    return existing_docs

In [ ]:
# extract all entries in vector store collection
existing_docs = get_all_entries(collection)

In [ ]:
def query_vector_database(VECTOR_STORE_PATH, COLLECTION_NAME, query, n=2):
    # query collection
    results = collection.query(
        query_texts=query,
        n_results=n
    )

    print(f"Similarity Search: {n} most similar entries:")
    print(results["documents"])
    return results

In [ ]:
# similarity search
similar_vector_entries = query_vector_database(VECTOR_STORE_PATH, COLLECTION_NAME, query=["Lilies are white."])